# ĐỀ XUẤT LAPTOP DỰA THEO NHU CẦU VÀ THÔNG TIN NGƯỜI DÙNG

Ở đề tài này, chúng em sẽ tạo ra một Recommendation System, dựa vào dữ liệu laptop trên thị trường và các số điểm đánh giá. Để đề xuất một danh sách các laptop phù hợp, với input là 1 **phân loại** cho trước (văn phòng, gaming, ...) và một số thông tin của người dùng **(nghề nghiệp, ngành học)**

## TRAIN DATA

Dữ liệu train sẽ được crawl từ 3 nguồn: 

- Tiki (tiki.vn)

- Nguyễn Kim (nguyenkim.com)

- Thế giới di động (thegioididong.com) 

In [146]:
%matplotlib inline
import matplotlib.pyplot as plt  
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.base import TransformerMixin
import re
import math

In [4]:
data = pd.read_csv('crawler/Merge.csv')

### Dữ liệu bao gồm các cột:

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1163 entries, 0 to 1162
Data columns (total 11 columns):
Brand         1163 non-null object
CPU           769 non-null object
Graphic       865 non-null object
Name          1163 non-null object
Price         1163 non-null int64
RAM           877 non-null object
SSD           1163 non-null int64
Score_star    1163 non-null float64
Type_CPU      829 non-null object
Vote          1163 non-null int64
Weight        722 non-null object
dtypes: float64(1), int64(3), object(7)
memory usage: 100.1+ KB


In [7]:
data.head(5)

,Brand,CPU,Graphic,Name,Price,RAM,SSD,Score_star,Type_CPU,Vote,Weight
0,HP,Intel Core i3,Intel HD Graphics 620,Laptop HP 240 G7 (9FM95PA),8990000,4 GB,0,4.9,7020U,10,NaN
1,Asus,NaN,NaN,Laptop Asus X409UA-EK092T,9390000,4 GB,0,4.4,NaN,10,1.6
2,Asus,Intel Core i5,Intel UHD Graphics 620,Laptop Asus X409FA (EK100T),11590000,4 GB,0,4.5,8265U,10,1.6
3,HP,Core i5,Intel UHD Graphics 620,Laptop HP NoteBook 14S-CF1040TU (7PU14PA),12990000,4 GB,0,4.6,8250U,10,NaN
4,HP,Intel Core i5,Intel HD Graphics 620,Laptop HP 348 G5 (7XU21PA),13990000,4 GB,1,4.5,8265U,10,1.48


## PHƯƠNG PHÁP ĐÁNH GIÁ VÀ CÁCH TIẾN HÀNH

Chúng em sẽ chọn phương pháp **supervised learning** để phần loại (tự cho yếu tố phân loại) và đề xuất (**user-based recommendation**) dựa theo thông tin matching giữa người dùng và sản phẩm.

### Yếu tố phân loại

Chúng em sẽ tạo 2 bảng heuristic để tính điểm cho mỗi sản phẩm - cột **points** là điểm sẽ tăng lên nếu sản phẩm match với **in** đã cho

In [125]:
cate_points = pd.read_csv('categories_points.csv')
major_points = pd.read_csv('major_points.csv')

In [173]:
cate_points.head()

,cate_id,name,field,in,points
0,0,van_phong,Price,"0,4000000",2
1,0,van_phong,Price,"4000000,10000000",1
2,1,gaming,Graphic,"nvidia,geforce,amd",3
3,1,gaming,RAM,"8,16,32",2
4,1,gaming,RAM,4,1


In [174]:
major_points.head()

,major,field,in,points
0,IT,RAM,"4,8,16",1


### Thuật toán cần chọn

> to be updating

### Tiền xử lý

1. Fill các giá trị trống

In [175]:
fill_string = ColumnTransformer([
    ('fill_empty', SimpleImputer(strategy='constant', fill_value='Unknown'), ['RAM', 'Name', 'Graphic'])
])

fill_nan = SimpleImputer(strategy='most_frequent')

preprocess = Pipeline([
    ('fill_string', fill_string),
    ('fill_nan', fill_nan),
])

2. Bỏ các chuỗi/đơn vị thừa

In [165]:
class UnitRemoverAndGraphicSimpler(TransformerMixin):
    def fit(self, *_):
        return self

    def transform(self, X_df: pd.DataFrame, *_) -> pd.DataFrame:
        df = X_df.copy()
        df['RAM'] = X_df['RAM'].apply(lambda ram: int( re.sub(r'\D', '', ram) ))
        df['Weight'] = X_df['Weight'].apply(lambda w: int( math.floor( float(re.sub(',', '.', w)) ) ))
        df['Graphic'] = X_df['Graphic'].apply(lambda g: g.split(' ')[0].lower())
        
        return df

3. Chuẩn hóa vào 1 pipeline

In [166]:
pipeline = Pipeline([
    ('preprocess', preprocess),
    ('onehot', OneHotEncoder()),
    ('kmean', KMeans(n_clusters=3, random_state=0, max_iter=500))
])

In [167]:
pipeline.fit(data)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

### TRAIN DATA

> updating

## KIỂM TRA VÀ DỰ ĐOÁN

> updating